In [9]:
# --- Data ---
circles = [
    ((9.5, 30.5), 3.5),
    ((11, 30.5), 3.5),
    ((18, 5), 5),
    ((4, 23), 3),
    ((4, 17), 4),
    ((7, 17), 4.5),
    ((12.5,17), 3),
    ((18, 15), 6),
    ((27,19),3),
    ((30,17),3),
    ((4, 11), 3),
    ((12, 9), 2),
    ((16,9),3),    
    ((6, 4),  3),
    ((7.5, 4), 2.5),
    ((18, 24), 5)
        ]

points = [
    (28, 18), 
    (9, 17), 
    (18, 9.5), 
    (18, 20), 
    (11, 5), 
    (14, 30.5)
        ]

In [10]:
def build_Q(circles, points):
    Q = []

    # 1) Input points
    for idx, (x, y) in enumerate(points, start=1):
        Q.append((x, y, "input", {idx}))

    # 2) Left endpoints of circles
    for idx, ((cx, cy), r) in enumerate(circles, start=1):
        left_x = cx - r
        left_y = cy
        Q.append((left_x, left_y, "left", {idx}))

    # 3) Order the list Q by x, then by y
    Q.sort(key=lambda t: (t[0], t[1]))

    return Q

In [14]:
Q = build_Q(circles, points)
print(Q)

[(0, 17, 'left', {5}), (1, 11, 'left', {11}), (1, 23, 'left', {4}), (2.5, 17, 'left', {6}), (3, 4, 'left', {14}), (5.0, 4, 'left', {15}), (6.0, 30.5, 'left', {1}), (7.5, 30.5, 'left', {2}), (9, 17, 'input', {2}), (9.5, 17, 'left', {7}), (10, 9, 'left', {12}), (11, 5, 'input', {5}), (12, 15, 'left', {8}), (13, 5, 'left', {3}), (13, 9, 'left', {13}), (13, 24, 'left', {16}), (14, 30.5, 'input', {6}), (18, 9.5, 'input', {3}), (18, 20, 'input', {4}), (24, 19, 'left', {9}), (27, 17, 'left', {10}), (28, 18, 'input', {1})]


In [85]:
import math

def in_circle(x, y, cx, cy, r):
    print((x-cx)**2 + (y-cy)**2 <= r**2 + 1e-12)
    return (x-cx)**2 + (y-cy)**2 <= r**2 + 1e-12

def satisfies(cond, x, y, circles, boundary):
    print(cond)
    # inside constraints
    for cid in cond["in"]:
        (cx, cy), r = circles[cid-1]
        if not in_circle(x, y, cx, cy, r): 
            return False
    # outside constraints
    for cid in cond["out"]:
        (cx, cy), r = circles[cid-1]
        if in_circle(x, y, cx, cy, r):
            return False
    # y-inequalities vs boundaries
    for var, op, (cid, which) in cond.get("ineq", []):
        y_ref = boundary[cid][which]
        print(y_ref, var, op, y)
        if op == "<" and not (y < y_ref): return False
        if op == ">" and not (y > y_ref): return False
    return True

In [86]:
# INF = float("inf")

# sweepL = {
#     "x": 0.0,                          # starting x (we’ll set later)
#     "segments": [(-INF, INF, set())],  # one initial region: whole vertical line empty
#     "active": set()                    # no active circles yet
# }

# print(sweepL)

eps = 1e-9

# Suppose Q is already built and sorted, e.g. [(5.0, 4.0, "left", {15}), ...]
x0 = Q[0][0] - eps

sweepL = {
    "x": x0,             # start just before the first event
    "active": set(),     # no active circles yet
    "boundary": {},      # will fill later
    "conditions": [
        {"in": set(), "out": set(), "ineq": []}  # whole line empty
    ]
}


In [87]:
import math

# def split_segment_for_new_circle(sweepL, circle_id, circles, eps=1e-6):
#     (cx, cy), r = circles[circle_id - 1]

#     # 1) evaluate just to the right of the event x
#     x_prime = sweepL["x"] + eps
#     print(x_prime)
#     dx = x_prime - cx
#     print(dx)
#     h = math.sqrt(max(r*r - dx*dx, 0.0))
#     print(h)
#     y_low, y_high = cy - h, cy + h

#     segs = sweepL["segments"]

#     # 2) find segment containing cy
#     k = None
#     for i, (ymin, ymax, S) in enumerate(segs):
#         if ymin < cy < ymax:
#             k = i
#             break
#     if k is None:
#         return  # nothing to do; cy lies on a boundary (rare numeric edge)

#     ymin, ymax, S = segs[k]

#     # 3) build the three replacement segments (skip empty ones)
#     new_segs = []
#     if ymin < y_low:
#         new_segs.append((ymin, y_low, S))
#     new_segs.append((y_low, y_high, S | {circle_id}))
#     if y_high < ymax:
#         new_segs.append((y_high, ymax, S))

#     # 4) replace in-place
#     sweepL["segments"] = segs[:k] + new_segs + segs[k+1:]

#     # 5) mark circle active
#     sweepL.setdefault("active", set()).add(circle_id)
#     print(sweepL)

#     # return y_low, y_high in case you want to log or draw
#     return y_low, y_high

import math

def split_condition_for_new_circle(sweepL, circle_id, circles, eps=1e-6):
    """
    At current sweepL['x'], insert circle `circle_id` into the conditions model:
      - compute y_low, y_high at x' = x + eps
      - find the condition that currently contains (x', cy)
      - replace that condition with three logical conditions:
          (down)   outside new circle AND y < y_low
          (middle) inside new circle
          (up)     outside new circle AND y > y_high
      - update boundary and active
    Returns (y_low, y_high).
    """
    (cx, cy), r = circles[circle_id - 1]

    # 1) Evaluate just to the right of the left endpoint to avoid tangency
    x_prime = sweepL["x"] + eps
    dx = x_prime - cx
    # If numerically outside, just activate and exit (nothing to split)
    if abs(dx) > r + 1e-15:
        sweepL.setdefault("active", set()).add(circle_id)
        return None, None

    h = math.sqrt(max(r*r - dx*dx, 0.0))
    y_low, y_high = cy - h, cy + h

    # 2) Update boundary for this circle at current x
    sweepL.setdefault("boundary", {})
    sweepL["boundary"][circle_id] = {"y_low": y_low, "y_high": y_high}

    # 3) Find the single condition that contains (x_prime, cy) *before* inserting the circle
    conds = sweepL["conditions"]
    k = None
    for i, cond in enumerate(conds):
        if satisfies(cond, x_prime, cy, circles, sweepL["boundary"]):
            k = i
            break
    if k is None:
        # Fallback: if nothing matched due to numeric edge, put it in the last condition
        k = len(conds) - 1

    base = conds[k]
    # Make shallow copies so we don't mutate the originals
    base_in  = set(base.get("in",  set()))
    base_out = set(base.get("out", set()))
    base_ineq = list(base.get("ineq", []))

    # 4) Build the three replacement conditions
    down = {
        "in":  set(base_in),
        "out": set(base_out) | {circle_id},
        "ineq": base_ineq + [("y", "<", (circle_id, "y_low"))]
    }
    middle = {
        "in":  set(base_in) | {circle_id},
        "out": set(base_out),
        "ineq": list(base_ineq)
    }
    up = {
        "in":  set(base_in),
        "out": set(base_out) | {circle_id},
        "ineq": base_ineq + [("y", ">", (circle_id, "y_high"))]
    }

    # 5) Replace the matched condition with [down, middle, up]
    sweepL["conditions"] = conds[:k] + [down, middle, up] + conds[k+1:]

    # 6) Mark circle active
    sweepL.setdefault("active", set()).add(circle_id)
    print(sweepL)

    # Optional: debug print
    # print("x'=", x_prime, "y_low,y_high=", y_low, y_high)
    # for c in sweepL["conditions"]:
    #     print(c)

    return sweepL


In [88]:
# split_segment_for_new_circle(sweepL, circle_id=5, circles=circles, eps=1e-6)
split_condition_for_new_circle(sweepL, circle_id=5, circles=circles, eps=1e-6)

{'in': set(), 'out': set(), 'ineq': []}
{'x': -1e-09, 'active': {5}, 'boundary': {5: {'y_low': 16.99717298761903, 'y_high': 17.00282701238097}}, 'conditions': [{'in': set(), 'out': {5}, 'ineq': [('y', '<', (5, 'y_low'))]}, {'in': {5}, 'out': set(), 'ineq': []}, {'in': set(), 'out': {5}, 'ineq': [('y', '>', (5, 'y_high'))]}]}


{'x': -1e-09,
 'active': {5},
 'boundary': {5: {'y_low': 16.99717298761903, 'y_high': 17.00282701238097}},
 'conditions': [{'in': set(), 'out': {5}, 'ineq': [('y', '<', (5, 'y_low'))]},
  {'in': {5}, 'out': set(), 'ineq': []},
  {'in': set(), 'out': {5}, 'ineq': [('y', '>', (5, 'y_high'))]}]}

In [89]:
def advance_sweep_to(sweepL, x_event, circles):
    sweepL["x"] = x_event
# Advance to circle 11's left endpoint
(cx, cy), r = circles[11 - 1]
advance_sweep_to(sweepL, cx - r, circles)

split_condition_for_new_circle(sweepL, circle_id=11, circles=circles, eps=1e-6)

{'in': set(), 'out': {5}, 'ineq': [('y', '<', (5, 'y_low'))]}
False
16.99717298761903 y < 11
{'x': 1, 'active': {11, 5}, 'boundary': {5: {'y_low': 16.99717298761903, 'y_high': 17.00282701238097}, 11: {'y_low': 10.997550510461188, 'y_high': 11.002449489538812}}, 'conditions': [{'in': set(), 'out': {11, 5}, 'ineq': [('y', '<', (5, 'y_low')), ('y', '<', (11, 'y_low'))]}, {'in': {11}, 'out': {5}, 'ineq': [('y', '<', (5, 'y_low'))]}, {'in': set(), 'out': {11, 5}, 'ineq': [('y', '<', (5, 'y_low')), ('y', '>', (11, 'y_high'))]}, {'in': {5}, 'out': set(), 'ineq': []}, {'in': set(), 'out': {5}, 'ineq': [('y', '>', (5, 'y_high'))]}]}


{'x': 1,
 'active': {5, 11},
 'boundary': {5: {'y_low': 16.99717298761903, 'y_high': 17.00282701238097},
  11: {'y_low': 10.997550510461188, 'y_high': 11.002449489538812}},
 'conditions': [{'in': set(),
   'out': {5, 11},
   'ineq': [('y', '<', (5, 'y_low')), ('y', '<', (11, 'y_low'))]},
  {'in': {11}, 'out': {5}, 'ineq': [('y', '<', (5, 'y_low'))]},
  {'in': set(),
   'out': {5, 11},
   'ineq': [('y', '<', (5, 'y_low')), ('y', '>', (11, 'y_high'))]},
  {'in': {5}, 'out': set(), 'ineq': []},
  {'in': set(), 'out': {5}, 'ineq': [('y', '>', (5, 'y_high'))]}]}

In [68]:
sweepL

{'x': -1e-09,
 'active': {5, 11},
 'boundary': {5: {'y_low': 16.99717298761903, 'y_high': 17.00282701238097}},
 'conditions': [{'in': set(), 'out': {5}, 'ineq': [('y', '<', (5, 'y_low'))]},
  {'in': {5}, 'out': set(), 'ineq': []},
  {'in': set(), 'out': {5}, 'ineq': [('y', '>', (5, 'y_high'))]}]}

In [39]:
def algoritmo(circles, points):
    Ac = circles
    Q = build_Q(circles, points)
    INF = float("inf")
    sweepL = {
        "x": 0.0,                          # starting x (we’ll set later)
        "segments": [(-INF, INF, set())],  # one initial region: whole vertical line empty
        "active": set()                    # no active circles yet
    }
    print("Antes de iniciar:")
    print("Q = ", Q)
    print("Línea de barrido =", sweepL)
    print("Ac =", Ac)
    i = 0
    while i < len(Q):
        x, y, kind, idx = Q[i]
    
        if kind == "input":
            print(f"Point ({x}, {y}) is an INPUT point, index {idx}")
    
        elif kind == "left":
            circle_idx = list(idx)[0]  # get the circle index from the set
            print(f"Point ({x}, {y}) is a LEFT endpoint of circle {idx}")

            # Actualizar L
            # split_condition_for_new_circle(sweepL, circle_id=circle_idxhyujjjjjjjjhyyyyyyyyyyyyyy, circles=circles, eps=1e-6)
            
            # compute right endpoint of the same circle
            cx, cy = circles[circle_idx - 1]  # -1 because circles is 0-based
            (cx, cy), r = circles[circle_idx - 1]
            right_x = cx + r
            right_y = cy
            Q.append((right_x, right_y, "right", {circle_idx}))
    
        elif kind == "right":
            print(f"Point ({x}, {y}) is a RIGHT endpoint of circle {idx}")
    
        i += 1




algoritmo(circles, points)

Antes de iniciar:
Q =  [(0, 17, 'left', {5}), (1, 11, 'left', {11}), (1, 23, 'left', {4}), (2.5, 17, 'left', {6}), (3, 4, 'left', {14}), (5.0, 4, 'left', {15}), (6.0, 30.5, 'left', {1}), (7.5, 30.5, 'left', {2}), (9, 17, 'input', {2}), (9.5, 17, 'left', {7}), (10, 9, 'left', {12}), (11, 5, 'input', {5}), (12, 15, 'left', {8}), (13, 5, 'left', {3}), (13, 9, 'left', {13}), (13, 24, 'left', {16}), (14, 30.5, 'input', {6}), (18, 9.5, 'input', {3}), (18, 20, 'input', {4}), (24, 19, 'left', {9}), (27, 17, 'left', {10}), (28, 18, 'input', {1})]
Línea de barrido = {'x': 0.0, 'segments': [(-inf, inf, set())], 'active': set()}
Ac = [((9.5, 30.5), 3.5), ((11, 30.5), 3.5), ((18, 5), 5), ((4, 23), 3), ((4, 17), 4), ((7, 17), 4.5), ((12.5, 17), 3), ((18, 15), 6), ((27, 19), 3), ((30, 17), 3), ((4, 11), 3), ((12, 9), 2), ((16, 9), 3), ((6, 4), 3), ((7.5, 4), 2.5), ((18, 24), 5)]
Point (0, 17) is a LEFT endpoint of circle {5}
5
Point (1, 11) is a LEFT endpoint of circle {11}
11
Point (1, 23) is a LEFT